In [1]:
import os
import pandas as pd
import numpy as np
import json
from unidecode import unidecode

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def normalize_value(value, is_date=False):
    if value != value:
        return ''

    if is_date:
        try:
            return date_parser.parse(value)
        except:
            try:
                return pd.to_datetime(value)
            except:
                pass    
        
    if type(value).__module__ == 'numpy':
        value = value.item()

    if isinstance(value, str):            
        if value.startswith('-') and value.replace('-', '').replace(',', '').isdigit():                
            return int(value.replace(',', ''))
        if value.replace(',', '').isdigit():                
            return int(value.replace(',', ''))
        if value.startswith('=') and value.isdigit():                
            return int(value.replace('=', ''))
        if value.startswith('-') and value.replace('-', '').replace(',', '').replace('.', '').isdigit():                
            return float(value.replace(',', ''))
        if value.replace(',', '').replace('.', '').isdigit():                
            return float(value.replace(',', ''))
        
        value = value.strip().lower()

        if value in ('none', 'n/a', 'nan', '-'):
            return '' 

        value = value.replace('&', 'and')

        if value == 'united states':
            return 'usa'
        if value == 'united kingdom':
            return 'uk'
        
        value = unidecode(value)        
        value = ''.join(c for c in value if c.isalnum()) 
        return value

    return value

def normalize_key(value, is_date=False):
    if value != value:
        return ''

    if is_date:
        try:
            return str(date_parser.parse(value))
        except:
            try:
                return str(pd.to_datetime(value))
            except:
                pass  

    if isinstance(value, str):  
        value = value.strip().lower()

        if value in ('none', 'n/a', 'nan', '-', '--', 'unknown'):
            return '' 

        value = value.replace('&', 'and')

        if value == 'united states':
            return 'usa'
        if value == 'united kingdom':
            return 'uk'

        value = unidecode(value)        
        value = ''.join(c for c in value if c.isalnum()) 
        return value

    return str(value)

def normalize_primary_columns(df, norm_columns, date_columns, primary_columns, keys_type):
    for col in norm_columns:
        df[col] = df[col].apply(normalize_key, col in date_columns)
    
    for col, key_type in zip(primary_columns, keys_type):
        if key_type == 'year':
            df[col] = df[col].astype(float).astype(int)
            
        df[col] = df[col].astype(str)
        
    return [tuple(r) for r in df[primary_columns].to_numpy()]    

def find_row(df, columns, values):
    query = ' & '.join([f'(`{col}`=="{value}")' for col, value in zip(columns, values)])    
    return df.query(query)                    

def exclude_row_and_cutoff(df, row_cond, cutoff_cond):
    if row_cond and cutoff_cond:
        query = " & ".join([row_cond, cutoff_cond])
        return df.query(query)
    
    if row_cond:
        return df.query(row_cond)
    
    if cutoff_cond:
        return df.query(cutoff_cond)

    return df

def evaluate_table(df_fetched, df_ref, primary_columns, keys_type, date_columns, epsilons, norm_example, norm_cutoff):
    cutoff = md['cutoff'] if norm_cutoff else {}
                     
    columns = df_ref.columns
    #df_fetched.columns = columns    
    df_fetched = df_fetched[columns]
    df_fetched = df_fetched.drop_duplicates(subset=primary_columns)   

    norm_columns = set(primary_columns)
    if cutoff and cutoff['type'] == 'between':
        norm_columns.update([cutoff['field']])
        
    for pc in primary_columns:
        df_fetched = df_fetched[df_fetched[pc].notna()]    
    
    fetched_entities = normalize_primary_columns(df_fetched, norm_columns, date_columns, primary_columns, keys_type)
    ref_entities = normalize_primary_columns(df_ref, norm_columns, date_columns, primary_columns, keys_type)

    cutoff_cond = None
    if cutoff and cutoff['type'] == 'between':
        field = cutoff['field']
        start = cutoff['start']
        end = cutoff['end']
        cutoff_cond = f'({field}>="{start}" & {field} <="{end}")'  
        
    row_cond = None
    if norm_example:
        exclude_vals = ref_entities[0]
        row_cond = 'not (' + ' & '.join([f'(`{col}`=="{val}")' for col, val in zip(primary_columns, exclude_vals)]) + ')'

        if exclude_vals in fetched_entities:
            fetched_entities.remove(exclude_vals)
        ref_entities.remove(exclude_vals)
        
    df_ref = exclude_row_and_cutoff(df_ref, row_cond, cutoff_cond)
    df_fetched = exclude_row_and_cutoff(df_fetched, row_cond, cutoff_cond)

    total_matches = 0
    key_matches = 0

    for fetched_entity in fetched_entities:
        if fetched_entity in ref_entities: 
            row_fetched = find_row(df_fetched, primary_columns, fetched_entity)
            row_ref = find_row(df_ref, primary_columns, fetched_entity)
            key_matches += 1
            
            for column in columns:
                try:
                    value_fetched = row_fetched[column].values[0]
                    value_ref = row_ref[column].values[0]

                    norm_value_fetched = normalize_value(value_fetched, column in date_columns)
                    norm_value_ref = normalize_value(value_ref, column in date_columns)

                    if norm_value_fetched == norm_value_ref:    
                        total_matches += 1
                    elif column in epsilons and norm_value_ref != '' and norm_value_fetched != '':
                        if norm_value_ref * 0.999 < norm_value_fetched < norm_value_ref * 1.001:
                            total_matches += 1   
                except:
                    pass
            
    recall = total_matches/(df_ref.shape[0] * df_ref.shape[1])
    precision = total_matches/(df_fetched.shape[0] * df_fetched.shape[1])
    f1_score = 2*recall*precision/(recall+precision) if (recall + precision) > 0 else 0.0

    keys_recall = key_matches/len(ref_entities)
    keys_precision = key_matches/len(fetched_entities)
    keys_f1_score = 2*keys_recall*keys_precision/(keys_recall+keys_precision) if (keys_recall + keys_precision) > 0 else 0.0
    
    nk = len(primary_columns)
    
    non_keys_recall = (total_matches - key_matches*nk) / (df_ref.shape[0] * (df_ref.shape[1] - nk))
    non_keys_precision = (total_matches - key_matches*nk) / (df_fetched.shape[0] * (df_fetched.shape[1] - nk))
    non_keys_f1_score = 2*non_keys_recall*non_keys_precision/(non_keys_recall+non_keys_precision) if (non_keys_recall + non_keys_precision) > 0 else 0.0
     
    relative_non_key_accuracy = (total_matches - key_matches*nk) / (key_matches * (df_ref.shape[1] - nk))    
        
    return keys_recall, keys_precision, keys_f1_score, non_keys_recall, non_keys_precision, non_keys_f1_score, recall, precision, f1_score, relative_non_key_accuracy

## JSON

In [56]:
norm_example = True
norm_cutoff = False
exp_name = 'gpt_4_turbo_w_break_full_table_first_example_20240928-182411'

In [57]:
result_folder = 'DATA/%s' % exp_name
tables_folder = 'DATA/%s/Tables' % exp_name

metadata_path="DATA/Benchmark/cfg.json"

with open(metadata_path, "rb") as f:
    metadata = json.load(f)

In [58]:
tables = []

keys_recall_scores = []
keys_precision_scores = []
keys_f1_scores = []
non_keys_recall_scores = []
non_keys_precision_scores = []
non_keys_f1_scores = []
recall_scores = []
precision_scores = []
f1_scores = []
rel_nk_acc_scores = []

for i in range(100):        
    idx = "%d" % i
    md = metadata[idx]
    print(md['name'])
    try:
        fetched_table = pd.read_csv(os.path.join(tables_folder, "%s.csv" % md['name']))
        gt_table = pd.read_csv(md['path'])
        primary_columns = md['keys']
        keys_type = md['keys_type']
        date_columns = md['dateColumns']
        epsilons = md['epsilons']
        kr, kp, kf1, nkr, nkp, nkf1, r, p, f1, rnka  = evaluate_table(fetched_table, 
                                                                       gt_table, 
                                                                       primary_columns, 
                                                                       keys_type,
                                                                       date_columns, 
                                                                       epsilons, 
                                                                       norm_example, 
                                                                       norm_cutoff)
    except:    
        kr, kp, kf1, nkr, nkp, nkf1, r, p, f1, rnka = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
        
    tables.append(md['name'])
    
    keys_recall_scores.append(kr)
    keys_precision_scores.append(kp)
    keys_f1_scores.append(kf1)
    non_keys_recall_scores.append(nkr)
    non_keys_precision_scores.append(nkp)
    non_keys_f1_scores.append(nkf1)
    recall_scores.append(r)
    precision_scores.append(p)
    f1_scores.append(f1)
    rel_nk_acc_scores.append(rnka)
    
    print("====================")
    print("Keys Recall: %.4f" % kr)
    print("Keys Precision: %.4f" % kp)
    print("Keys F1: %.4f" % kf1)
    
    print("Non Keys Recall: %.4f" % nkr)
    print("Non Keys Precision: %.4f" % nkp)
    print("Non Keys F1: %.4f" % nkf1)
    
    print("Relative Non Keys Accuracy: %.4f" % rnka)
    
    print("Recall: %.4f" % r)
    print("Precision: %.4f" % p)
    print("F1: %.4f" % f1)    
    
    print("====================")    
    
res_df = pd.DataFrame([tables, 
                       keys_recall_scores,
                       keys_precision_scores,
                       keys_f1_scores,
                       non_keys_recall_scores,
                       non_keys_precision_scores,
                       non_keys_f1_scores,
                       rel_nk_acc_scores,
                       recall_scores,
                       precision_scores, 
                       f1_scores                       
                       ]).T
res_df.columns = ['Table', 
                  'Keys_Recall', 'Keys_Precision', 'Keys_F1_Score',
                  'Non_Keys_Recall', 'Non_Keys_Precision', 'Non_Keys_F1_Score', 'Rel_Non_Keys_Accuracy',
                  'Recall', 'Precision', 'F1_Score'
                  ]

res_df['Keys_Recall'] = res_df['Keys_Recall'].astype(float).round(4)
res_df['Keys_Precision'] = res_df['Keys_Precision'].astype(float).round(4)
res_df['Keys_F1_Score'] = res_df['Keys_F1_Score'].astype(float).round(4)
res_df['Non_Keys_Recall'] = res_df['Non_Keys_Recall'].astype(float).round(4)
res_df['Non_Keys_Precision'] = res_df['Non_Keys_Precision'].astype(float).round(4)
res_df['Non_Keys_F1_Score'] = res_df['Non_Keys_F1_Score'].astype(float).round(4)
res_df['Rel_Non_Keys_Accuracy'] = res_df['Rel_Non_Keys_Accuracy'].astype(float).round(4)
res_df['Recall'] = res_df['Recall'].astype(float).round(4)
res_df['Precision'] = res_df['Precision'].astype(float).round(4)
res_df['F1_Score'] = res_df['F1_Score'].astype(float).round(4)

means = pd.DataFrame(['All'] + res_df.mean(axis=0, numeric_only=True).tolist()).T
means.columns = res_df.columns        
res_df = pd.concat([res_df, means], axis=0)

fname = "scores%s%s" % ("_norm_example" if norm_example else "", "_post_cutoff" if norm_cutoff else "")
res_df.to_csv("%s/%s_%d.csv" % (result_folder, fname, len(res_df)-1), index=False)

republican_straw_polls_2012
Keys Recall: 0.0000
Keys Precision: 0.0000
Keys F1: 0.0000
Non Keys Recall: 0.0000
Non Keys Precision: 0.0000
Non Keys F1: 0.0000
Relative Non Keys Accuracy: 0.0000
Recall: 0.0000
Precision: 0.0000
F1: 0.0000
russia_demographics_1946_2012
Keys Recall: 0.2121
Keys Precision: 1.0000
Keys F1: 0.3500
Non Keys Recall: 0.0498
Non Keys Precision: 0.2347
Non Keys F1: 0.0821
Relative Non Keys Accuracy: 0.2347
Recall: 0.0606
Precision: 0.2857
F1: 0.1000
belgium_demographics_1900_2011
Keys Recall: 0.3333
Keys Precision: 1.0000
Keys F1: 0.5000
Non Keys Recall: 0.0000
Non Keys Precision: 0.0000
Non Keys F1: 0.0000
Relative Non Keys Accuracy: 0.0000
Recall: 0.0370
Precision: 0.1111
F1: 0.0556
australia_demographics_1900_2010
Keys Recall: 0.1000
Keys Precision: 1.0000
Keys F1: 0.1818
Non Keys Recall: 0.0091
Non Keys Precision: 0.0909
Non Keys F1: 0.0165
Relative Non Keys Accuracy: 0.0909
Recall: 0.0192
Precision: 0.1919
F1: 0.0349
new_brunswick_parishes_2006_2011
Keys Reca

Keys Recall: 0.2000
Keys Precision: 0.8065
Keys F1: 0.3205
Non Keys Recall: 0.2000
Non Keys Precision: 0.8065
Non Keys F1: 0.3205
Relative Non Keys Accuracy: 1.0000
Recall: 0.2000
Precision: 0.8065
F1: 0.3205
ramsar_convention_parties
Keys Recall: 0.5818
Keys Precision: 0.9796
Keys F1: 0.7300
Non Keys Recall: 0.1667
Non Keys Precision: 0.2806
Non Keys F1: 0.2091
Relative Non Keys Accuracy: 0.2865
Recall: 0.3051
Precision: 0.5136
F1: 0.3828
guitar_hero_5_songs
Keys Recall: 0.5119
Keys Precision: 0.9149
Keys F1: 0.6565
Non Keys Recall: 0.2143
Non Keys Precision: 0.3830
Non Keys F1: 0.2748
Relative Non Keys Accuracy: 0.4186
Recall: 0.2639
Precision: 0.4716
F1: 0.3384
south_cambridgeshire_district_council_1973_2012
Keys Recall: 0.2821
Keys Precision: 1.0000
Keys F1: 0.4400
Non Keys Recall: 0.0912
Non Keys Precision: 0.3232
Non Keys F1: 0.1422
Relative Non Keys Accuracy: 0.3232
Recall: 0.1103
Precision: 0.3909
F1: 0.1720
dublin_maternity_hospital_mortality_rates_1784_1849
Keys Recall: 1.000

Keys Recall: 0.9048
Keys Precision: 1.0000
Keys F1: 0.9500
Non Keys Recall: 0.0238
Non Keys Precision: 0.0263
Non Keys F1: 0.0250
Relative Non Keys Accuracy: 0.0263
Recall: 0.2000
Precision: 0.2211
F1: 0.2100
tulsa_shock_2010
Keys Recall: 0.5294
Keys Precision: 1.0000
Keys F1: 0.6923
Non Keys Recall: 0.0267
Non Keys Precision: 0.0505
Non Keys F1: 0.0350
Relative Non Keys Accuracy: 0.0505
Recall: 0.0686
Precision: 0.1296
F1: 0.0897
london_heathrow_busiest_routes_2012
Keys Recall: 0.6610
Keys Precision: 0.6610
Keys F1: 0.6610
Non Keys Recall: 0.0226
Non Keys Precision: 0.0226
Non Keys F1: 0.0226
Relative Non Keys Accuracy: 0.0342
Recall: 0.1822
Precision: 0.1822
F1: 0.1822
hungarian_grand_prix_qualifying_2012
Keys Recall: 1.0000
Keys Precision: 1.0000
Keys F1: 1.0000
Non Keys Recall: 0.6377
Non Keys Precision: 0.6377
Non Keys F1: 0.6377
Relative Non Keys Accuracy: 0.6377
Recall: 0.6894
Precision: 0.6894
F1: 0.6894
farum_park_national_games
Keys Recall: 0.0526
Keys Precision: 0.1000
Keys 

Keys Recall: 1.0000
Keys Precision: 1.0000
Keys F1: 1.0000
Non Keys Recall: 1.0000
Non Keys Precision: 1.0000
Non Keys F1: 1.0000
Relative Non Keys Accuracy: 1.0000
Recall: 1.0000
Precision: 1.0000
F1: 1.0000
through_the_wormhole_season_4
Keys Recall: 1.0000
Keys Precision: 1.0000
Keys F1: 1.0000
Non Keys Recall: 0.5000
Non Keys Precision: 0.5000
Non Keys F1: 0.5000
Relative Non Keys Accuracy: 0.5000
Recall: 0.6667
Precision: 0.6667
F1: 0.6667
un_habitat_scroll_of_honour_award_1991
Keys Recall: 0.0000
Keys Precision: 0.0000
Keys F1: 0.0000
Non Keys Recall: 0.0000
Non Keys Precision: 0.0000
Non Keys F1: 0.0000
Relative Non Keys Accuracy: 0.0000
Recall: 0.0000
Precision: 0.0000
F1: 0.0000
miss_universe_semifinal_scores_1993
Keys Recall: 0.7778
Keys Precision: 0.7778
Keys F1: 0.7778
Non Keys Recall: 0.0000
Non Keys Precision: 0.0000
Non Keys F1: 0.0000
Relative Non Keys Accuracy: 0.0000
Recall: 0.1556
Precision: 0.1556
F1: 0.1556
woodley_season_1_2012
Keys Recall: 0.0000
Keys Precision: 0

In [59]:
res_df

,Table,Keys_Recall,Keys_Precision,Keys_F1_Score,Non_Keys_Recall,Non_Keys_Precision,Non_Keys_F1_Score,Rel_Non_Keys_Accuracy,Recall,Precision,F1_Score
0,republican_straw_polls_2012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,russia_demographics_1946_2012,0.2121,1.0,0.35,0.0498,0.2347,0.0821,0.2347,0.0606,0.2857,0.1
2,belgium_demographics_1900_2011,0.3333,1.0,0.5,0.0,0.0,0.0,0.0,0.037,0.1111,0.0556
3,australia_demographics_1900_2010,0.1,1.0,0.1818,0.0091,0.0909,0.0165,0.0909,0.0192,0.1919,0.0349
4,new_brunswick_parishes_2006_2011,0.0397,0.6667,0.075,0.0,0.0,0.0,0.0,0.0114,0.1905,0.0214
...,...,...,...,...,...,...,...,...,...,...,...
96,through_the_wormhole_season_4,1.0,1.0,1.0,0.5,0.5,0.5,0.5,0.6667,0.6667,0.6667
97,un_habitat_scroll_of_honour_award_1991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,miss_universe_semifinal_scores_1993,0.7778,0.7778,0.7778,0.0,0.0,0.0,0.0,0.1556,0.1556,0.1556
99,woodley_season_1_2012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
